# CS4248 NLP Project Team 18

## Installing necessary libraries

*Note*: Put libraries that need to be installed with `!pip install LIBRARY` so that we can ensure consistency in packages

In [2]:
!pip install scikit-learn

  Using cached scikit_learn-1.4.1.post1-cp311-cp311-win_amd64.whl (10.6 MB)
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl (15.8 MB)
                                              0.0/46.2 MB ? eta -:--:--
                                              1.1/46.2 MB 34.7 MB/s eta 0:00:02
     ---                                      3.6/46.2 MB 45.8 MB/s eta 0:00:01
     -----                                    6.6/46.2 MB 52.9 MB/s eta 0:00:01
     --------                                 9.4/46.2 MB 54.4 MB/s eta 0:00:01
     ----------                              13.0/46.2 MB 65.6 MB/s eta 0:00:01
     -------------                           15.8/46.2 MB 65.2 MB/s eta 0:00:01
     ---------------                         18.9/46.2 MB 65.2 MB/s eta 0:00:01
     ------------------                      22.0/46.2 MB 65.6 MB/s eta 0:00:01
     -------------------                     23.0/46.2 MB 65.6 MB/s eta 0:00:01
     ---------------------                   25.4/46.2 MB 50.4 MB/s 


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Data Analysis

## Preprocessing

## Feature Engineering

## Model Training & Evaluation

### Logistic Regression

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

## Note: Change this to fit the algorithm below
# X are the features
X = np.array([[10,2], [2,3], [4,6], [5,1], [12,4], [14,5]])
# y are the outputs
y = np.array([1, 0, 0, 0, 1, 1])
# test_size is the size of the test (0 < test_size < 1)
test_size = 0.2
# seed for random split
seed = 40
## End of Note

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
model = LogisticRegression()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

evaluation_metrics = [("Accuracy", accuracy_score), ("F1 score", f1_score), ("Confusion Matrix", confusion_matrix)]

for evaluation_metric_name, evaluation_metric_func in evaluation_metrics:
    print(f"{evaluation_metric_name}:\n{evaluation_metric_func(y_test, y_pred)}")
    

Accuracy:
1.0
F1 score:
1.0
Confusion Matrix:
[[1 0]
 [0 1]]
